<a href="https://colab.research.google.com/github/srspratti/nlp_proj_1/blob/main/Proj1_Baseline_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction : Import and Mount Google Drive with Data

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import csv
import pandas_profiling
from pandas_profiling import ProfileReport
import time

In [ ]:
!pwd
!fusermount -u drive/

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
#!pip install pandas_profiling==2.9.0

In [4]:
ls /content/

drive/  sample_data/


In [5]:
ls /content/drive/My\ Drive/Colab-Notebooks/data/proj-1/train_1/train/0A5RMYN.female.36.Sagittarius.csv

'/content/drive/My Drive/Colab-Notebooks/data/proj-1/train_1/train/0A5RMYN.female.36.Sagittarius.csv'


# Run the below git commands to push the code to github

In [33]:
!git add .
!git commit -m "Added classifier"
!git push -u origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


Reading any single csv file in the train dataset to explore

In [ ]:
sample_df=pd.read_csv("/content/drive/My Drive/Colab-Notebooks/data/proj-1/train_1/train/0A5RMYN.female.36.Sagittarius.csv", encoding="UTF-8",sep=",", engine='python', header=None, names=["ID","Gender","Age","Zodiac Sign","Blog-post"])

In [ ]:
sample_df.iloc[:,4]

In [ ]:
sample_df['Zodiac Sign']
sample_df.iloc[1,0]
sample_df.iloc[:,4]
sample_df['ID']

writing the sample dataframe as csv to explore further if req.

In [ ]:
sample_df.to_csv("/content/drive/My Drive/Colab-Notebooks/data/proj-1/sample_df_3.csv")

In [ ]:
!ls -l /content/drive/My\ Drive/Colab-Notebooks/data/proj-1/train_1/train | wc -l # Printing the # of csv files for train in the directory 

# Converting all csv files ( bloggers and posts) into a single Master Data Frame ( 'Age' , 'Gender', 'Zodiac Sign' , 'Blog-post')

---



In [ ]:
#@title
!pwd

In [6]:
#@title
import glob
import os
path = r'/content/drive/My Drive/Colab-Notebooks/data/proj-1/train_1/train/'
all_csv_files = glob.glob(os.path.join(path, "*.csv"))

In [ ]:
#@title
len(all_csv_files)

In [7]:
#@title
csv.field_size_limit(sys.maxsize)

131072

In [ ]:
#@title
df_each_csv_file = []
df_from_each_file = (pd.read_csv(each_csv_file, error_bad_lines=False,sep=',', encoding="UTF-8", engine='python', header=None, names=["ID","Gender","Age","Zodiac Sign","Blog-post"]) for each_csv_file in all_csv_files)
df_all_csv_file   = pd.concat(df_from_each_file, ignore_index=True, axis=0)

NameError: ignored

##**Writing the Master DataFrame to a CSV** 

In [ ]:
#@title
df_all_csv_file.to_csv("/content/drive/My Drive/Colab-Notebooks/data/proj-1/train_1/df_all_csv_file_2.csv", index=False)

# Data Exploration 

In [ ]:
len(df_all_csv_file)

In [ ]:
len(df_master_train)

In [ ]:
df_all_csv_file.info()

In [ ]:
df_master_train.info()

In [ ]:
df_all_csv_file.head(5)

In [ ]:
df_master_train.head(5)

In [ ]:
df_all_csv_file.loc()

In [ ]:
len(df_all_csv_file.columns)

In [ ]:
len(df_all_csv_file.rows)

In [ ]:
df_all_csv_file.describe()

## More!! Exploring the master dataframe using Pandas_Profiling

In [ ]:
profile = df_all_csv_file.profile_report(title='Blog Sample report')

In [ ]:
profile.to_file("/content/drive/My Drive/Colab-Notebooks/data/proj-1/df_all_csv_file_output.html")

# Splitting this dataset into three dataframes to build Age, Gender and Zodiac_Sign Classifiers

In [8]:
df_master_train = pd.read_csv("/content/drive/My Drive/Colab-Notebooks/data/proj-1/train_1/df_all_csv_file_2.csv", sep=',', encoding="UTF-8", engine='python')
df_gender = df_master_train.iloc[:,[1,4]]
df_age = df_master_train.iloc[:,[2,4]]
df_zodiacSign = df_master_train.iloc[:,[3,4]]

# Gender Classifiers

df_gender

```
# This is formatted as code
```



In [ ]:
df_gender.describe

Exploring male blog-posts


In [ ]:
df_gender_male = df_gender[df_gender['Gender'] == 'male']
df_gender_male
len(df_gender_male)

Exploring female blog-posts

In [ ]:
df_gender_female = df_gender[df_gender['Gender'] == 'female']
df_gender_female
#len(df_gender_female)

## Naive Bayes

In [ ]:
print("The below given code will demonstrate the Naive Bayes Classifier")

1. To run a Naive Bayes classifier in Scikit Learn, the categories must be numeric. Here, we assign 0 to male and 1 to female 

In [14]:
df_gender['label'] = df_gender['Gender'].apply(lambda x: 0 if x=='male' else 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_gender[df_gender['Gender']=='female']

In [ ]:
df_gender[df_gender['Gender']=='male']

2. Split data into training and testing sets 

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_gender['Blog-post'], df_gender['label'], random_state=1)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

3. Word Count Vectors

In [17]:
start=time.time()
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)
elapsed=time.time()
print(elapsed-start)

73.54530429840088


4. Model Fitting

In [18]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_cv, y_train)
predictions = naive_bayes.predict(X_test_cv)

5. Results : Accuracy Score , Prediction Score and Recall Score

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print('Accuracy score: ', accuracy_score(y_test, predictions))
print('Precision score: ', precision_score(y_test, predictions))
print('Recall score: ', recall_score(y_test, predictions))

Accuracy score:  0.6785249044174495
Precision score:  0.6398128504318836
Recall score:  0.7486700948665662


        Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
cm = confusion_matrix(y_test, predictions)
sns.heatmap(cm, square=True, annot=True, cmap='RdBu', cbar=False,
xticklabels=['female', 'male'], yticklabels=['female', 'male'])
plt.xlabel('true label')
plt.ylabel('predicted label')

Text(91.68, 0.5, 'predicted label')

6. Investigating the model's misses

In [21]:
testing_predictions = []
for i in range(len(X_test)):
    if predictions[i] == 1:
        testing_predictions.append('male')
    else:
        testing_predictions.append('female')
check_df = pd.DataFrame({'actual_label': list(y_test), 'prediction': testing_predictions, 'Blog-post':list(X_test)})
check_df.replace(to_replace=0, value='female', inplace=True)
check_df.replace(to_replace=1, value='male', inplace=True)

In [ ]:
check_df

###### Data Frame of the word count vectors ( for further data exploration)

In [ ]:
word_freq_df = pd.DataFrame(X_train_cv.toarray(), columns=cv.get_feature_names())
top_words_df = pd.DataFrame(word_freq.sum()).sort_values(0, ascending=False)

## SVM

In [24]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

1. Model Fitting

In [27]:
type(X_train_cv)
X_train_cv_small.shape

(283262, 403012)

In [28]:
X_train_cv_small=X_train_cv[:10000]
y_train_small=y_train[:10000]
X_test_cv_small=X_test_cv[:10000]
y_test_small=y_test[:10000]

In [30]:
start_time=time.time()
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', verbose=True)
SVM.fit(X_train_cv_small,y_train_small)
elapsed_time=time.time()
print(elapsed_time-start_time)

[LibSVM]196.6964032649994


2. Model Prediction

In [31]:
predictions_SVM = SVM.predict(X_test_cv_small)

3. Results : Accuracy

In [32]:
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test_small)*100)

SVM Accuracy Score ->  59.68


# Age Classifiers


In [ ]:
df_age

In [ ]:
set(df_age['Age'])

In [ ]:
df_age1 = df_age.copy

lambda x : x*2 if x < 10 else (x*3 if x < 20 else x)

In [ ]:
df_age['label'] = df_age['Age'].apply(lambda x: 0 if (int(x)>=0 and int(x)<=19) else ( 1 if (int(x)>=20 and int(x)<=29) else 2 ) )

In [ ]:
df_age[df_age['label']==2]

  2. Split data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(df_age['Blog-post'], df_age['label'], random_state=1)

len(X_train_age)

len(X_test_age)

3. Word Count Vectors

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')
X_train_age_cv = cv.fit_transform(X_train_age)
X_test_age_cv = cv.transform(X_test_age)

4. Model Fitting

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_age_cv, y_train_age)
predictions = naive_bayes.predict(X_test_age_cv)

5. Results : Accuracy Score, Prediction Score and Recall Score

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print('Accuracy score: ', accuracy_score(y_test_age, predictions))
#print('Precision score: ', precision_score(y_test_age, predictions))
#print('Recall score: ', recall_score(y_test_age, predictions))

Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
cm = confusion_matrix(y_test, predictions)
sns.heatmap(cm, square=True, annot=True, cmap='RdBu', cbar=False,
xticklabels=['Age[0-19]', 'Age[20-29]', 'Age[30+]'], yticklabels=['Age[0-19]', 'Age[20-29]', 'Age[30+]'])
plt.xlabel('true label')
plt.ylabel('predicted label')

# Zodiac Sign Classifiers


In [ ]:
df_zodiacSign['Zodiac Sign'].head(100)

### Naive Bayes Classifier

Assigning Labels

In [ ]:
#df_zodiacSign['label'] = df_zodiacSign['Zodiac Sign'].apply(lambda x: 0 if x=='Aquarius' else ( 1 if x=='Aries' else ( 2 if x=='Cancer') else (3 if x=='Capricorn') else (4 if x=='Gemini') else (5 if x=='Leo') else 6 )
df_zodiacSign['label'] = df_zodiacSign['Zodiac Sign'].apply(lambda x: 0 if x=='Aquarius' else ( 1 if x=='Aries' else ( 2 if x=='Cancer' else (3 if x=='Capricorn' else ( 4 if x=='Gemini' else ( 5 if x=='Leo' else( 6 if x=='Libra' else ( 7 if x=='Pisces' else ( 8 if x=='Sagittarius' else ( 9 if x=='Scorpio' else ( 10 if x=='Taurus' else 11)))))))))))

In [ ]:
df_ZodiacSign_Aquarius=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Aquarius']

In [ ]:
df_ZodiacSign_Aries=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Aries']

In [ ]:
df_ZodiacSign_Cancer=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Cancer']

In [ ]:
df_ZodiacSign_Capricorn=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Capricorn']

In [ ]:
df_ZodiacSign_Gemini=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Gemini']

In [ ]:
df_ZodiacSign_Leo=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Leo']

In [ ]:
df_ZodiacSign_Libra=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Libra']

In [ ]:
df_ZodiacSign_Pisces=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Pisces']

In [ ]:
df_ZodiacSign_Sagittarius=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Sagittarius']

In [ ]:
df_ZodiacSign_Scorpio=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Scorpio']

In [ ]:
df_ZodiacSign_Taurus=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Taurus']

In [ ]:
df_ZodiacSign_Virgo=df_zodiacSign[df_zodiacSign['Zodiac Sign']=='Virgo']

2. Split data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train_Zodiac, X_test_Zodiac, y_train_Zodiac, y_test_Zodiac = train_test_split(df_gender['Blog-post'], df_zodiacSign['label'], random_state=1)

3. Word Count Vectors

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')
X_train_Zodiac_cv = cv.fit_transform(X_train_Zodiac)
X_test_Zodiac_cv = cv.transform(X_test_Zodiac)

4. Model Fitting

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_Zodiac_cv, y_train_Zodiac)
predictions = naive_bayes.predict(X_test_Zodiac_cv)

5. Results : Accuracy

In [ ]:

from sklearn.metrics import accuracy_score, precision_score, recall_score
print('Accuracy score: ', accuracy_score(y_test_Zodiac, predictions))
#print('Precision score: ', precision_score(y_test, predictions))
#print('Recall score: ', recall_score(y_test, predictions))